## Notebook to visualize the viewer functionality


In [1]:
from notebook_imports import *

import time
import torch
from torch.utils.data import DataLoader

from pyrad.data.dataloader import TrainDataloader
from pyrad.data.image_dataset import ImageDataset, PanopticImageDataset
from pyrad.data.image_sampler import CacheImageSampler
from pyrad.data.pixel_sampler import PixelSampler
from pyrad.data.utils import DatasetInputs, get_dataset_inputs_from_dataset_config
from pyrad.graphs.modules.ray_generator import RayGenerator
from pyrad.graphs.modules.scene_colliders import SceneBoundsCollider, AABBBoxCollider
from pyrad.cameras.cameras import get_camera
from pyrad.cameras.camera_paths import InterpolatedCameraPath
from pyrad.cameras.rays import RayBundle
from pyrad.utils.io import get_absolute_path
from pyrad.utils.plotly import get_line_segments_from_lines
from pyrad.cameras.cameras import get_camera_model
from pyrad.utils.misc import get_dict_to_torch, instantiate_from_dict_config

# the new import
from pyrad.viewer.backend import vis_utils


from hydra import compose, initialize
from omegaconf import open_dict
import pprint
from tqdm import tqdm
import random

In [2]:
# establish the connection to the tcp backend, to talk to the visualizer
# zmq_url should match the output of `python run_zmq_server.py`
vis = vis_utils.get_vis(zmq_url="tcp://127.0.0.1:6000")

In [3]:
vis.delete()

# draws a red box in the scene
vis_utils.show_box_test(vis)

In [4]:
data = vis["/Cameras/Main Camera"].get_object()

In [5]:
data

b'\x83\xa4type\xaaset_object\xa4path\xb3Cameras/Main Camera\xa6object\x82\xa8metadata\x83\xa7version\xcb@\x12\x00\x00\x00\x00\x00\x00\xa4type\xa6Object\xa9generator\xafObject3D.toJSON\xa6object\x8c\xa4uuid\xd9$9a332925-1e54-436a-a6d5-56dc185a39ba\xa4type\xb1PerspectiveCamera\xa6layers\x01\xa6matrix\xdc\x00\x10\xcb?\xe6\xa0\x9ef\x7f;\xcc\xcb\xbc\x80\x00\x00\x00\x00\x00\x00\xcb?\xe6\xa0\x9ef\x7f;\xcc\x00\xcb\xbf\xda \xbdp\x0c,<\xcb\xbf\xea \xbdp\x0c,<\xcb?\xda \xbdp\x0c,=\x00\xcb?\xe2y\xa7E\x903\x1c\xcb\xbf\xe2y\xa7E\x903\x1c\xcb\xbf\xe2y\xa7E\x903\x1a\x00\x05\xfb\xfb\x01\xa3fovx\xa4zoom\x01\xa4near\xcb?\x84z\xe1G\xae\x14{\xa3fard\xa5focus\n\xa6aspect\xcb?\xf6\x9df8\x15\xb5\xd3\xa9filmGauge#\xaafilmOffset\x00'

In [6]:
import umsgpack

In [7]:
m = umsgpack.unpackb(data)

In [8]:
from pyrad.viewer.backend.utils import get_chunks, get_intrinsics_matrix_and_camera_to_world_h

In [9]:
camera_object = m["object"]["object"]
intrinsics_matrix, camera_to_world_h = get_intrinsics_matrix_and_camera_to_world_h(camera_object, image_height=100)

In [10]:
intrinsics_matrix

array([[28.86751346,  0.        , 70.67137809],
       [ 0.        , 28.86751346, 50.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [11]:
camera_to_world_h

array([[ 7.07106781e-01, -4.08248290e-01,  5.77350269e-01,
         5.00000000e+00],
       [-2.77555756e-17, -8.16496581e-01, -5.77350269e-01,
        -5.00000000e+00],
       [ 7.07106781e-01,  4.08248290e-01, -5.77350269e-01,
        -5.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [12]:
import numpy as np
image = np.random.rand(100, 100, 3)

In [13]:
image.shape

(100, 100, 3)

In [14]:
vis["/Cameras/Main Camera"].set_image(image)

b'ok'

In [42]:
camera_to_world_h.shape

torch.Size([4, 4])

In [36]:
fov

120

In [37]:
aspect

1.4134275618374559

In [34]:
type(camera_to_world_h)

list

In [29]:
m["object"]

{'metadata': {'version': 4.5,
  'type': 'Object',
  'generator': 'Object3D.toJSON'},
 'object': {'uuid': '8b429d04-9f7f-4461-b1bb-9bdd5ff6748c',
  'type': 'PerspectiveCamera',
  'castShadow': True,
  'receiveShadow': True,
  'layers': 1,
  'matrix': [0.5620843398756875,
   -0.31160318996883307,
   -0.7661361803672765,
   0,
   -0.3924307317037816,
   0.7149266753901418,
   -0.5786864173540215,
   0,
   0.7280517259652277,
   0.6259259547397327,
   0.27956642055548614,
   0,
   6.305112899549926,
   5.420677776926363,
   2.421116222461348,
   1],
  'fov': 120,
  'zoom': 1,
  'near': 0.01,
  'far': 100,
  'focus': 10,
  'aspect': 1.4134275618374559,
  'filmGauge': 35,
  'filmOffset': 0}}

In [10]:
with initialize(version_base=None, config_path="../configs"):
    config = compose(config_name="graph_default.yaml")

In [11]:
dataset_inputs = get_dataset_inputs_from_dataset_config(**config.data.dataset_inputs_train, split="train")

In [12]:
# ImageDataset
image_dataset_train = instantiate_from_dict_config(config.data.image_dataset_train, **dataset_inputs.as_dict())

In [13]:
# grab 10 images and show their frustums
indices = random.sample(range(len(image_dataset_train)), k=10)
for idx in indices:
    image = image_dataset_train[idx]["image"]
    camera = get_camera(dataset_inputs.intrinsics[idx], dataset_inputs.camera_to_world[idx], None)
    pose = camera.get_camera_to_world().double().numpy()
    K = camera.get_intrinsics_matrix().double().numpy()
    vis_utils.draw_camera_frustum(
        vis,
        image=(image.double().numpy() * 255.0),
        pose=pose,
        K=K,
        height=1.0,
        name="{:06d}".format(idx),
        displayed_focal_length=0.5,
        realistic=False,
    )

In [14]:
# move a camera around
num_cameras = len(dataset_inputs.intrinsics)
intrinsics = dataset_inputs.intrinsics
camera_to_world = dataset_inputs.camera_to_world
idx0, idx1 = random.sample(range(num_cameras), k=2)
camera_a = get_camera(intrinsics[idx0], camera_to_world[idx0], None)
camera_b = get_camera(intrinsics[idx1], camera_to_world[idx1], None)

num_steps = 100
fps = 30
estimated_time = num_steps / fps
print("estimated_time:", estimated_time)

camera_path = InterpolatedCameraPath(camera_a, camera_b)
cameras = camera_path.get_path(steps=num_steps)

start = time.time()
for camera in cameras:
    vis_utils.set_camera(vis, camera)
    time.sleep(1 / fps)
time_elapsed = time.time() - start
print("time_elapsed:", time_elapsed)

estimated_time: 3.3333333333333335
time_elapsed: 3.54646897315979
